# Libraries

In [ ]:
# Install only the packages not in Colab by default
!pip install -q grad-cam kaggle

In [ ]:
# Core ML libraries
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, random_split
from torchvision import transforms
import timm

# Grad-CAM for interpretability
from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from pytorch_grad_cam.utils.image import show_cam_on_image

# Data processing libraries
import numpy as np
import pandas as pd
from PIL import Image
import cv2

# Visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns

# Utility libraries
import os
import shutil
from pathlib import Path
import warnings
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split
import json
from datetime import datetime

# Hugging Face libraries
import transformers
from transformers import AutoImageProcessor
from datasets import Dataset as HFDataset

import zipfile
import requests

from google.colab import drive, files

# System Check

In [ ]:
# Quick system check
print(f"PyTorch: {torch.__version__} | CUDA: {torch.cuda.is_available()}")
print(f"TIMM: {timm.__version__} | Transformers: {transformers.__version__}")

# Create project structure
os.makedirs('data/authentic', exist_ok=True)
os.makedirs('data/tampered', exist_ok=True)

free_gb = shutil.disk_usage("/")[2] // (2**30)
print(f"Available space: {free_gb} GB")

# Data Pipeline

In [ ]:
def setup_drive():
    """Mount Drive and create project structure"""
    print("Mounting Google Drive...")
    drive.mount('/content/drive')

    # Create project directory in Drive
    project_dir = '/content/drive/MyDrive/fraud_detection'
    os.makedirs(project_dir, exist_ok=True)
    os.makedirs(f'{project_dir}/models', exist_ok=True)
    os.makedirs(f'{project_dir}/outputs', exist_ok=True)
    os.makedirs(f'{project_dir}/data_splits', exist_ok=True)

    print(f"Project directory created: {project_dir}")
    return project_dir

def check_colab_resources():
    """Verify Colab environment setup"""
    print("=== Colab Environment Check ===")
    print(f"PyTorch version: {torch.__version__}")
    print(f"CUDA available: {torch.cuda.is_available()}")

    if torch.cuda.is_available():
        gpu_name = torch.cuda.get_device_name(0)
        print(f"GPU: {gpu_name}")
        print(f"GPU memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

    # Check disk space
    total, used, free = shutil.disk_usage('/')
    print(f"Disk space: {free / 1e9:.1f} GB free")

    return torch.cuda.is_available()

def setup_kaggle_colab():
    """Setup Kaggle API in Colab with file upload"""
    print("Setting up Kaggle API...")

    # Install Kaggle
    os.system("pip install kaggle -q")

    # Upload kaggle.json
    print("Please upload your kaggle.json file:")
    uploaded = files.upload()

    if 'kaggle.json' in uploaded:
        # Setup credentials
        os.makedirs('/root/.kaggle', exist_ok=True)
        shutil.move('kaggle.json', '/root/.kaggle/kaggle.json')
        os.chmod('/root/.kaggle/kaggle.json', 0o600)
        print("Kaggle credentials configured successfully")
        return True
    else:
        print("kaggle.json not uploaded")
        return False

def download_casia_optimized():
    """Download CASIA dataset optimized for Colab performance"""

    # Create temp directories for fast processing
    os.makedirs('/content/data/authentic', exist_ok=True)
    os.makedirs('/content/data/tampered', exist_ok=True)

    print("Downloading CASIA dataset to temp storage...")

    # Download to temp storage
    download_cmd = "kaggle datasets download -d divg07/casia-20-image-tampering-detection-dataset -p /content/"
    result = os.system(download_cmd)

    if result != 0:
        print("Download failed")
        return False

    # Extract dataset
    print("Extracting dataset...")
    zip_files = list(Path('/content/').glob('*.zip'))
    if not zip_files:
        print("No zip file found")
        return False

    with zipfile.ZipFile(zip_files[0], 'r') as zip_ref:
        zip_ref.extractall('/content/raw')

    # Find and organize
    print("Organizing dataset...")
    raw_dir = Path('/content/raw')

    # Search for Au and Tp folders
    au_folder = None
    tp_folder = None

    for item in raw_dir.rglob('*'):
        if item.is_dir() and item.name == 'Au':
            au_folder = item
        elif item.is_dir() and item.name == 'Tp':
            tp_folder = item

    if not au_folder or not tp_folder:
        print("Could not find Au/Tp folders")
        return False

    # Copy to organized structure
    authentic_count = 0
    tampered_count = 0

    print("Copying authentic images...")
    for img in au_folder.glob('*.jpg'):
        shutil.copy(img, '/content/data/authentic/')
        authentic_count += 1

    print("Copying tampered images...")
    for img in tp_folder.glob('*.jpg'):
        shutil.copy(img, '/content/data/tampered/')
        tampered_count += 1

    # Cleanup temp files
    os.remove(zip_files[0])
    shutil.rmtree('/content/raw')

    print(f"Dataset ready: {authentic_count} authentic, {tampered_count} tampered")
    return True

def create_train_splits(project_dir):
    """Create train/val/test splits and save to Drive"""
    print("Creating data splits...")

    # Get file lists
    authentic_files = list(Path('/content/data/authentic').glob('*.jpg'))
    tampered_files = list(Path('/content/data/tampered').glob('*.jpg'))

    # Create labels
    data = []
    for f in authentic_files:
        data.append({'file_path': str(f), 'label': 0, 'class': 'authentic'})
    for f in tampered_files:
        data.append({'file_path': str(f), 'label': 1, 'class': 'tampered'})

    df = pd.DataFrame(data)

    # Split data
    train_df, temp_df = train_test_split(df, test_size=0.4, stratify=df['label'], random_state=42)
    val_df, test_df = train_test_split(temp_df, test_size=0.5, stratify=temp_df['label'], random_state=42)

    # Save splits to Drive
    splits_dir = f'{project_dir}/data_splits'
    train_df.to_csv(f'{splits_dir}/train_split.csv', index=False)
    val_df.to_csv(f'{splits_dir}/val_split.csv', index=False)
    test_df.to_csv(f'{splits_dir}/test_split.csv', index=False)

    print(f"Data splits created:")
    print(f"  Train: {len(train_df)} images")
    print(f"  Validation: {len(val_df)} images")
    print(f"  Test: {len(test_df)} images")
    print(f"Splits saved to Google Drive")

    return train_df, val_df, test_df

def complete_colab_setup():
    """Run complete setup for Colab environment"""
    print("Starting Colab setup for fraud detection project...")
    print("=" * 50)

    # Step 1: Check resources
    has_gpu = check_colab_resources()
    if not has_gpu:
        print("WARNING: No GPU detected. Enable GPU in Runtime -> Change runtime type")
        return False

    # Step 2: Setup Drive
    project_dir = setup_drive()

    # Step 3: Setup Kaggle
    if not setup_kaggle_colab():
        return False

    # Step 4: Download dataset
    if not download_casia_optimized():
        return False

    # Step 5: Create splits
    train_df, val_df, test_df = create_train_splits(project_dir)

    print("\nSetup complete! Ready for model training.")
    print(f"Dataset: {len(train_df) + len(val_df) + len(test_df)} total images")
    print(f"Working directory: /content/data/")
    print(f"Results will be saved to: {project_dir}")

    return True

def verify_setup():
    """Verify everything is ready"""
    auth_count = len(list(Path('/content/data/authentic').glob('*.jpg')))
    tamp_count = len(list(Path('/content/data/tampered').glob('*.jpg')))

    print(f"Dataset verification:")
    print(f"  Authentic: {auth_count}")
    print(f"  Tampered: {tamp_count}")
    print(f"  GPU available: {torch.cuda.is_available()}")

    # Check if splits exist in Drive
    project_dir = '/content/drive/MyDrive/fraud_detection'
    splits_exist = all([
        Path(f'{project_dir}/data_splits/train_split.csv').exists(),
        Path(f'{project_dir}/data_splits/val_split.csv').exists(),
        Path(f'{project_dir}/data_splits/test_split.csv').exists()
    ])
    print(f"  Data splits ready: {splits_exist}")

    return auth_count > 0 and tamp_count > 0 and torch.cuda.is_available()

# Data Loading

In [ ]:
# Run complete Colab setup
if complete_colab_setup():
    verify_setup()
    print("Ready to start training!")
else:
    print("Setup failed - check the steps above")

# Training Pipeline

In [ ]:
# EfficientNet Training Pipeline for Fraud Detection

class FraudDataset(Dataset):
    """Custom dataset for fraud detection images"""

    def __init__(self, dataframe, transform=None):
        self.data = dataframe
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        image_path = row['file_path']
        label = row['label']

        # Load image
        image = Image.open(image_path).convert('RGB')

        if self.transform:
            image = self.transform(image)

        return image, label

def create_transforms():
    """Create training and validation transforms"""

    # Training transforms with augmentation
    train_transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomRotation(degrees=15),
        transforms.ColorJitter(brightness=0.2, contrast=0.2),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                           std=[0.229, 0.224, 0.225])
    ])

    # Validation/test transforms (no augmentation)
    val_transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                           std=[0.229, 0.224, 0.225])
    ])

    return train_transform, val_transform

def create_model(num_classes=2):
    """Create EfficientNet-B0 model for fraud detection"""

    # Load pre-trained EfficientNet-B0
    model = timm.create_model('efficientnet_b0.ra_in1k',
                             pretrained=True,
                             num_classes=num_classes)

    print(f"Model created: EfficientNet-B0")
    print(f"Total parameters: {sum(p.numel() for p in model.parameters()):,}")
    print(f"Trainable parameters: {sum(p.numel() for p in model.parameters() if p.requires_grad):,}")

    return model

def create_data_loaders(batch_size=32):
    """Create data loaders for training, validation, and testing"""

    # Load data splits
    project_dir = '/content/drive/MyDrive/fraud_detection'
    train_df = pd.read_csv(f'{project_dir}/data_splits/train_split.csv')
    val_df = pd.read_csv(f'{project_dir}/data_splits/val_split.csv')
    test_df = pd.read_csv(f'{project_dir}/data_splits/test_split.csv')

    # Create transforms
    train_transform, val_transform = create_transforms()

    # Create datasets
    train_dataset = FraudDataset(train_df, transform=train_transform)
    val_dataset = FraudDataset(val_df, transform=val_transform)
    test_dataset = FraudDataset(test_df, transform=val_transform)

    # Calculate class weights for imbalanced dataset
    class_counts = train_df['label'].value_counts().sort_index()
    total_samples = len(train_df)
    class_weights = [total_samples / (2 * count) for count in class_counts]
    print(f"Class distribution: {class_counts.values}")
    print(f"Class weights: {class_weights}")

    # Create data loaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size,
                             shuffle=True, num_workers=2, pin_memory=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size,
                           shuffle=False, num_workers=2, pin_memory=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size,
                            shuffle=False, num_workers=2, pin_memory=True)

    print(f"Data loaders created:")
    print(f"  Train batches: {len(train_loader)}")
    print(f"  Val batches: {len(val_loader)}")
    print(f"  Test batches: {len(test_loader)}")

    return train_loader, val_loader, test_loader, class_weights

def train_model(model, train_loader, val_loader, class_weights, num_epochs=15):
    """Train the fraud detection model"""

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)

    # Loss function with class weights for imbalanced data
    weights = torch.FloatTensor(class_weights).to(device)
    criterion = nn.CrossEntropyLoss(weight=weights)

    # Optimizer with learning rate scheduling
    optimizer = optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-4)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min',
                                                    factor=0.5, patience=3, verbose=True)

    # Training history
    history = {
        'train_loss': [],
        'train_acc': [],
        'val_loss': [],
        'val_acc': []
    }

    best_val_acc = 0.0
    best_model_state = None

    print(f"Starting training on {device}")
    print("=" * 50)

    for epoch in range(num_epochs):
        # Training phase
        model.train()
        train_loss = 0.0
        train_correct = 0
        train_total = 0

        for batch_idx, (images, labels) in enumerate(train_loader):
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            train_total += labels.size(0)
            train_correct += (predicted == labels).sum().item()

            # Print progress every 50 batches
            if batch_idx % 50 == 0:
                print(f'Epoch {epoch+1}/{num_epochs}, Batch {batch_idx}/{len(train_loader)}, '
                      f'Loss: {loss.item():.4f}')

        # Validation phase
        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0

        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)

                val_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()

        # Calculate metrics
        train_loss_avg = train_loss / len(train_loader)
        train_acc = 100 * train_correct / train_total
        val_loss_avg = val_loss / len(val_loader)
        val_acc = 100 * val_correct / val_total

        # Update history
        history['train_loss'].append(train_loss_avg)
        history['train_acc'].append(train_acc)
        history['val_loss'].append(val_loss_avg)
        history['val_acc'].append(val_acc)

        # Learning rate scheduling
        scheduler.step(val_loss_avg)

        # Save best model
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_model_state = model.state_dict().copy()

        print(f'Epoch {epoch+1}/{num_epochs}:')
        print(f'  Train Loss: {train_loss_avg:.4f}, Train Acc: {train_acc:.2f}%')
        print(f'  Val Loss: {val_loss_avg:.4f}, Val Acc: {val_acc:.2f}%')
        print(f'  Best Val Acc: {best_val_acc:.2f}%')
        print('-' * 30)

    # Load best model (this ensures we're using the best performing epoch)
    model.load_state_dict(best_model_state)

    # Save model and training history with timestamp
    project_dir = '/content/drive/MyDrive/fraud_detection'
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

    # Save model state dict with metadata
    torch.save({
        'model_state_dict': best_model_state,
        'model_config': 'efficientnet_b0.ra_in1k',
        'num_classes': 2,
        'best_val_acc': best_val_acc,
        'class_weights': class_weights,
        'timestamp': timestamp
    }, f'{project_dir}/models/fraud_detection_model_{timestamp}.pth')

    # Save complete model for SageMaker deployment (this is the key addition you were missing)
    torch.save(model, f'{project_dir}/models/fraud_detection_complete_{timestamp}.pth')

    # Save training history
    with open(f'{project_dir}/outputs/training_history_{timestamp}.json', 'w') as f:
        json.dump(history, f, indent=2)

    print(f"Training completed!")
    print(f"Best validation accuracy: {best_val_acc:.2f}%")
    print(f"Model saved to Google Drive")
    print(f"Complete model (SageMaker ready): fraud_detection_complete_{timestamp}.pth")

    return model, history

def plot_training_history(history):
    """Plot training and validation metrics"""

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

    # Plot loss
    ax1.plot(history['train_loss'], label='Train Loss')
    ax1.plot(history['val_loss'], label='Val Loss')
    ax1.set_title('Training and Validation Loss')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Loss')
    ax1.legend()
    ax1.grid(True)

    # Plot accuracy
    ax2.plot(history['train_acc'], label='Train Accuracy')
    ax2.plot(history['val_acc'], label='Val Accuracy')
    ax2.set_title('Training and Validation Accuracy')
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Accuracy (%)')
    ax2.legend()
    ax2.grid(True)

    plt.tight_layout()

    # Save plot with timestamp
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    plt.savefig(f'/content/drive/MyDrive/fraud_detection/outputs/training_curves_{timestamp}.png',
                dpi=300, bbox_inches='tight')
    plt.show()

# Setup training pipeline
def setup_training():
    """Initialize everything for training"""
    print("Setting up training pipeline...")

    # Create data loaders
    train_loader, val_loader, test_loader, class_weights = create_data_loaders(batch_size=32)

    # Create model
    model = create_model(num_classes=2)

    return model, train_loader, val_loader, test_loader, class_weights

# Model Training

In [ ]:
# Start the training
model, train_loader, val_loader, test_loader, class_weights = setup_training()
model, history = train_model(model, train_loader, val_loader, class_weights, num_epochs=15)
plot_training_history(history)

# Testing Pipeline

In [ ]:
def load_best_model(timestamp=None):
    """Load the best trained model for testing"""
    project_dir = '/content/drive/MyDrive/fraud_detection'

    if timestamp is None:
        # Find most recent model if no timestamp provided
        model_files = list(Path(f'{project_dir}/models').glob('fraud_detection_complete_*.pth'))
        if not model_files:
            print("No trained models found")
            return None
        # Get most recent model
        model_path = max(model_files, key=os.path.getctime)
        timestamp = model_path.stem.split('_')[-2] + '_' + model_path.stem.split('_')[-1]
    else:
        model_path = f'{project_dir}/models/fraud_detection_complete_{timestamp}.pth'

    print(f"Loading model: {model_path}")
    model = torch.load(model_path, map_location='cpu', weights_only=False)

    # Move to GPU if available
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)
    model.eval()

    print(f"Model loaded successfully on {device}")
    return model, timestamp

def evaluate_on_test_set(model, test_loader):
    """Comprehensive evaluation on test set"""
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.eval()

    all_predictions = []
    all_labels = []
    all_probabilities = []
    correct_predictions = 0
    total_samples = 0

    print("Evaluating model on test set...")

    with torch.no_grad():
        for batch_idx, (images, labels) in enumerate(test_loader):
            images, labels = images.to(device), labels.to(device)

            # Get model outputs
            outputs = model(images)
            probabilities = torch.softmax(outputs, dim=1)
            _, predicted = torch.max(outputs, 1)

            # Store results
            all_predictions.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            all_probabilities.extend(probabilities.cpu().numpy())

            # Calculate running accuracy
            correct_predictions += (predicted == labels).sum().item()
            total_samples += labels.size(0)

            if batch_idx % 20 == 0:
                print(f'Test batch {batch_idx}/{len(test_loader)} processed')

    # Calculate final metrics
    test_accuracy = correct_predictions / total_samples

    print(f"\nTest Set Results:")
    print(f"Test Accuracy: {test_accuracy:.4f} ({test_accuracy*100:.2f}%)")
    print(f"Total samples: {total_samples}")
    print(f"Correct predictions: {correct_predictions}")

    return all_predictions, all_labels, all_probabilities, test_accuracy

def generate_detailed_metrics(predictions, labels, probabilities, timestamp):
    """Generate comprehensive evaluation metrics and save results"""
    from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve
    import matplotlib.pyplot as plt
    import seaborn as sns

    # Classification report
    class_names = ['Authentic', 'Tampered']
    report = classification_report(labels, predictions, target_names=class_names, output_dict=True)

    print("\nDetailed Classification Report:")
    print(classification_report(labels, predictions, target_names=class_names))

    # Confusion matrix
    cm = confusion_matrix(labels, predictions)

    # Plot confusion matrix
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=class_names, yticklabels=class_names)
    plt.title('Confusion Matrix - Fraud Detection Model')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')

    # Save confusion matrix
    project_dir = '/content/drive/MyDrive/fraud_detection'
    plt.savefig(f'{project_dir}/outputs/confusion_matrix_{timestamp}.png',
                dpi=300, bbox_inches='tight')
    plt.show()

    # ROC curve and AUC
    probabilities_array = np.array(probabilities)
    tampered_probs = probabilities_array[:, 1]  # Probabilities for tampered class

    auc_score = roc_auc_score(labels, tampered_probs)
    fpr, tpr, _ = roc_curve(labels, tampered_probs)

    plt.figure(figsize=(8, 6))
    plt.plot(fpr, tpr, linewidth=2, label=f'ROC Curve (AUC = {auc_score:.3f})')
    plt.plot([0, 1], [0, 1], 'k--', linewidth=1, label='Random Classifier')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve - Fraud Detection Model')
    plt.legend(loc="lower right")
    plt.grid(True)

    # Save ROC curve
    plt.savefig(f'{project_dir}/outputs/roc_curve_{timestamp}.png',
                dpi=300, bbox_inches='tight')
    plt.show()

    # Save detailed results
    results = {
        'test_accuracy': float(np.mean(np.array(predictions) == np.array(labels))),
        'auc_score': float(auc_score),
        'classification_report': report,
        'confusion_matrix': cm.tolist(),
        'timestamp': timestamp,
        'total_samples': len(labels)
    }

    with open(f'{project_dir}/outputs/test_evaluation_{timestamp}.json', 'w') as f:
        json.dump(results, f, indent=2)

    print(f"\nAUC Score: {auc_score:.4f}")
    print(f"Results saved to: {project_dir}/outputs/")

    return results

def analyze_prediction_confidence(predictions, labels, probabilities):
    """Analyze model confidence in predictions"""
    probabilities_array = np.array(probabilities)
    max_probs = np.max(probabilities_array, axis=1)

    # Separate correct and incorrect predictions
    correct_mask = np.array(predictions) == np.array(labels)
    correct_confidence = max_probs[correct_mask]
    incorrect_confidence = max_probs[~correct_mask]

    print(f"\nPrediction Confidence Analysis:")
    print(f"Correct predictions - Mean confidence: {np.mean(correct_confidence):.4f}")
    print(f"Incorrect predictions - Mean confidence: {np.mean(incorrect_confidence):.4f}")
    print(f"High confidence correct (>0.9): {np.sum(correct_confidence > 0.9)} / {len(correct_confidence)}")
    print(f"High confidence incorrect (>0.9): {np.sum(incorrect_confidence > 0.9)} / {len(incorrect_confidence)}")

    # Plot confidence distribution
    plt.figure(figsize=(10, 6))
    plt.hist(correct_confidence, bins=30, alpha=0.7, label='Correct Predictions', color='green')
    plt.hist(incorrect_confidence, bins=30, alpha=0.7, label='Incorrect Predictions', color='red')
    plt.xlabel('Prediction Confidence')
    plt.ylabel('Frequency')
    plt.title('Distribution of Prediction Confidence')
    plt.legend()
    plt.grid(True)
    plt.show()

def test_individual_samples(model, test_loader, num_samples=8):
    """Test model on individual samples and show predictions"""
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.eval()

    # Get a batch of test data
    images, labels = next(iter(test_loader))
    images, labels = images.to(device), labels.to(device)

    with torch.no_grad():
        outputs = model(images)
        probabilities = torch.softmax(outputs, dim=1)
        _, predicted = torch.max(outputs, 1)

    # Plot sample predictions
    fig, axes = plt.subplots(2, 4, figsize=(16, 8))
    axes = axes.ravel()

    class_names = ['Authentic', 'Tampered']

    for i in range(min(num_samples, len(images))):
        # Denormalize image for display
        img = images[i].cpu()
        mean = torch.tensor([0.485, 0.456, 0.406]).view(3, 1, 1)
        std = torch.tensor([0.229, 0.224, 0.225]).view(3, 1, 1)
        img = img * std + mean
        img = torch.clamp(img, 0, 1)

        # Convert to numpy and transpose
        img_np = img.permute(1, 2, 0).numpy()

        axes[i].imshow(img_np)

        true_label = class_names[labels[i].item()]
        pred_label = class_names[predicted[i].item()]
        confidence = probabilities[i].max().item()

        color = 'green' if predicted[i] == labels[i] else 'red'
        axes[i].set_title(f'True: {true_label}\nPred: {pred_label}\nConf: {confidence:.3f}',
                         color=color, fontsize=10)
        axes[i].axis('off')

    plt.tight_layout()
    plt.suptitle('Sample Predictions from Test Set', fontsize=16)
    plt.show()

# Testing Results

In [ ]:
# Load the best model (using your timestamp)
model, timestamp = load_best_model('20250603_145722')

# Create test data loader if not already available
if 'test_loader' not in locals():
    print("Creating test data loader...")
    _, _, test_loader, _ = create_data_loaders(batch_size=32)

# Run comprehensive evaluation
print("Starting comprehensive model evaluation...")
print("=" * 50)

predictions, labels, probabilities, test_accuracy = evaluate_on_test_set(model, test_loader)

# Analysis & Visualisation

In [ ]:
# Generate detailed metrics and visualizations
results = generate_detailed_metrics(predictions, labels, probabilities, timestamp)

# Analyze prediction confidence
analyze_prediction_confidence(predictions, labels, probabilities)

# Test individual samples
test_individual_samples(model, test_loader, num_samples=8)

print("\nModel testing completed!")
print(f"Test accuracy: {test_accuracy*100:.2f}%")
print(f"All results saved with timestamp: {timestamp}")

In [ ]:
def print_performance_summary(results):
    """Print comprehensive performance summary"""
    print("\n" + "="*60)
    print("FRAUD DETECTION MODEL - PERFORMANCE SUMMARY")
    print("="*60)

    print(f"Model Timestamp: {results['timestamp']}")
    print(f"Test Accuracy: {results['test_accuracy']*100:.2f}%")
    print(f"AUC Score: {results['auc_score']:.4f}")
    print(f"Total Test Samples: {results['total_samples']}")

    # Extract metrics from classification report
    report = results['classification_report']

    print(f"\nPer-Class Performance:")
    print(f"Authentic Images:")
    print(f"  Precision: {report['Authentic']['precision']:.4f}")
    print(f"  Recall: {report['Authentic']['recall']:.4f}")
    print(f"  F1-Score: {report['Authentic']['f1-score']:.4f}")

    print(f"Tampered Images:")
    print(f"  Precision: {report['Tampered']['precision']:.4f}")
    print(f"  Recall: {report['Tampered']['recall']:.4f}")
    print(f"  F1-Score: {report['Tampered']['f1-score']:.4f}")

    print(f"\nOverall Metrics:")
    print(f"  Macro Avg F1: {report['macro avg']['f1-score']:.4f}")
    print(f"  Weighted Avg F1: {report['weighted avg']['f1-score']:.4f}")

    # Confusion matrix analysis
    cm = np.array(results['confusion_matrix'])
    tn, fp, fn, tp = cm.ravel()

    print(f"\nConfusion Matrix Breakdown:")
    print(f"  True Negatives (Authentic correctly identified): {tn}")
    print(f"  False Positives (Authentic misclassified as Tampered): {fp}")
    print(f"  False Negatives (Tampered misclassified as Authentic): {fn}")
    print(f"  True Positives (Tampered correctly identified): {tp}")

    print("\n" + "="*60)

# Print comprehensive summary
print_performance_summary(results)

# Grad-CAM Implementation

In [ ]:
import cv2
import numpy as np
from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from pytorch_grad_cam.utils.image import show_cam_on_image
import matplotlib.pyplot as plt
import torch.nn.functional as F
from PIL import Image

class FraudDetectionExplainer:
    """
    Grad-CAM explainer for fraud detection model
    Analyzes what features the model focuses on for tampering detection
    """

    def __init__(self, model, target_layers=None):
        """
        Initialize explainer with trained model

        Args:
            model: Trained PyTorch model
            target_layers: Target layers for Grad-CAM (auto-detected if None)
        """
        self.model = model
        self.device = next(model.parameters()).device

        # Auto-detect target layer for EfficientNet if not specified
        if target_layers is None:
            target_layers = self._find_target_layers()

        self.grad_cam = GradCAM(model=model, target_layers=target_layers)

        # Standard preprocessing transforms
        self.transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                               std=[0.229, 0.224, 0.225])
        ])

        print(f"Fraud Detection Explainer initialized on {self.device}")
        print(f"Target layers: {[type(layer).__name__ for layer in target_layers]}")

    def _find_target_layers(self):
        """Find the best target layer for EfficientNet Grad-CAM"""
        # For EfficientNet, use the last convolutional layer
        target_layers = []
        for name, module in self.model.named_modules():
            if isinstance(module, torch.nn.Conv2d):
                target_layers = [module]

        print(f"Auto-detected target layer: {type(target_layers[0]).__name__}")
        return target_layers

    def explain_prediction(self, image_tensor, true_label=None, target_class=None):
        """
        Generate Grad-CAM explanation for a single image

        Args:
            image_tensor: Preprocessed image tensor
            true_label: Ground truth label (for display)
            target_class: Class to explain (None for predicted class)

        Returns:
            Dictionary with prediction and explanation results
        """
        self.model.eval()

        # Get model prediction
        with torch.no_grad():
            output = self.model(image_tensor.unsqueeze(0).to(self.device))
            probabilities = F.softmax(output, dim=1)
            predicted_class = torch.argmax(probabilities, dim=1).item()
            confidence = probabilities[0][predicted_class].item()

        # Determine target for explanation
        explain_class = target_class if target_class is not None else predicted_class
        targets = [ClassifierOutputTarget(explain_class)]

        # Generate Grad-CAM heatmap
        input_tensor = image_tensor.unsqueeze(0).to(self.device)
        grayscale_cam = self.grad_cam(input_tensor=input_tensor, targets=targets)
        grayscale_cam = grayscale_cam[0, :]  # Remove batch dimension

        # Prepare original image for visualization
        original_image = self._tensor_to_image(image_tensor)

        # Create CAM overlay
        cam_visualization = show_cam_on_image(original_image, grayscale_cam, use_rgb=True)

        # Prepare results
        class_names = ['Authentic', 'Tampered']
        result = {
            'prediction': {
                'class': class_names[predicted_class],
                'class_id': predicted_class,
                'confidence': confidence,
                'probabilities': {
                    'authentic': float(probabilities[0][0]),
                    'tampered': float(probabilities[0][1])
                }
            },
            'explanation': {
                'target_class': class_names[explain_class],
                'target_class_id': explain_class,
                'heatmap_intensity': float(np.mean(grayscale_cam)),
                'max_activation': float(np.max(grayscale_cam))
            },
            'visualization': {
                'original_image': original_image,
                'heatmap': grayscale_cam,
                'cam_overlay': cam_visualization
            }
        }

        if true_label is not None:
            result['ground_truth'] = {
                'class': class_names[true_label],
                'class_id': true_label,
                'is_correct': predicted_class == true_label
            }

        return result

    def _tensor_to_image(self, tensor):
        """Convert normalized tensor to displayable image"""
        # Denormalize
        mean = torch.tensor([0.485, 0.456, 0.406]).view(3, 1, 1)
        std = torch.tensor([0.229, 0.224, 0.225]).view(3, 1, 1)

        denorm_tensor = tensor * std + mean
        denorm_tensor = torch.clamp(denorm_tensor, 0, 1)

        # Convert to numpy
        image_np = denorm_tensor.permute(1, 2, 0).numpy()
        return image_np.astype(np.float32)

def create_fraud_explainer(model_timestamp='20250603_145722'):
    """
    Create fraud detection explainer from saved model

    Args:
        model_timestamp: Timestamp of saved model

    Returns:
        FraudDetectionExplainer instance
    """
    print("Creating Grad-CAM explainer...")

    # Load the trained model
    model, _ = load_best_model(model_timestamp)

    # Create explainer
    explainer = FraudDetectionExplainer(model)

    return explainer

def visualize_explanation(result, figsize=(15, 5), save_path=None):
    """
    Create comprehensive visualization of Grad-CAM results

    Args:
        result: Result dictionary from explain_prediction
        figsize: Figure size tuple
        save_path: Optional save path
    """
    fig, axes = plt.subplots(1, 3, figsize=figsize)

    # Original image
    axes[0].imshow(result['visualization']['original_image'])
    axes[0].set_title('Original Image', fontsize=12)
    axes[0].axis('off')

    # Heatmap
    im = axes[1].imshow(result['visualization']['heatmap'], cmap='jet')
    axes[1].set_title(f"Grad-CAM Heatmap\n(Intensity: {result['explanation']['heatmap_intensity']:.3f})",
                      fontsize=12)
    axes[1].axis('off')
    plt.colorbar(im, ax=axes[1], fraction=0.046)

    # Overlay visualization
    axes[2].imshow(result['visualization']['cam_overlay'])

    # Create detailed title for overlay
    pred = result['prediction']
    exp = result['explanation']

    title = f"Prediction: {pred['class']} ({pred['confidence']:.3f})\n"
    title += f"Explaining: {exp['target_class']}"

    if 'ground_truth' in result:
        gt = result['ground_truth']
        correctness = "✓" if gt['is_correct'] else "✗"
        title += f"\nGround Truth: {gt['class']} {correctness}"

    axes[2].set_title(title, fontsize=12)
    axes[2].axis('off')

    plt.tight_layout()

    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        print(f"Visualization saved to: {save_path}")

    plt.show()

def analyze_tampering_focus(result):
    """
    Analyze what regions the model focuses on for tampering detection

    Args:
        result: Result dictionary from explain_prediction
    """
    heatmap = result['visualization']['heatmap']
    pred = result['prediction']

    # Calculate focus statistics
    high_activation_threshold = 0.7
    medium_activation_threshold = 0.4

    high_focus_ratio = np.sum(heatmap > high_activation_threshold) / heatmap.size
    medium_focus_ratio = np.sum(heatmap > medium_activation_threshold) / heatmap.size

    print(f"\nTampering Detection Focus Analysis:")
    print(f"Predicted Class: {pred['class']} (confidence: {pred['confidence']:.3f})")
    print(f"High activation regions (>{high_activation_threshold}): {high_focus_ratio:.2%}")
    print(f"Medium activation regions (>{medium_activation_threshold}): {medium_focus_ratio:.2%}")
    print(f"Average activation intensity: {result['explanation']['heatmap_intensity']:.3f}")
    print(f"Maximum activation intensity: {result['explanation']['max_activation']:.3f}")

    # Provide interpretation guidance
    if pred['class'] == 'Tampered':
        if high_focus_ratio > 0.1:
            print("Interpretation: Model detected strong tampering artifacts in multiple regions")
        elif medium_focus_ratio > 0.2:
            print("Interpretation: Model detected moderate tampering artifacts across image")
        else:
            print("Interpretation: Model detected subtle tampering artifacts")
    else:
        if high_focus_ratio < 0.05:
            print("Interpretation: Model found consistent authentic patterns throughout image")
        else:
            print("Interpretation: Model detected some suspicious regions but overall classified as authentic")

In [ ]:
def test_gradcam_samples(explainer, test_loader, num_samples=6):
    """
    Test Grad-CAM on sample images from test set

    Args:
        explainer: FraudDetectionExplainer instance
        test_loader: Test data loader
        num_samples: Number of samples to analyze
    """
    print("Testing Grad-CAM on sample images from test set...")
    print("=" * 60)

    # Get sample batch
    images, labels = next(iter(test_loader))

    for i in range(min(num_samples, len(images))):
        print(f"\nAnalyzing Sample {i+1}/{num_samples}")
        print("-" * 40)

        # Get explanation
        result = explainer.explain_prediction(images[i], true_label=labels[i].item())

        # Visualize
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        save_path = f'/content/drive/MyDrive/fraud_detection/outputs/gradcam_sample_{i+1}_{timestamp}.png'
        visualize_explanation(result, save_path=save_path)

        # Analyze focus patterns
        analyze_tampering_focus(result)

def test_gradcam_interesting_cases(explainer, test_loader):
    """
    Test Grad-CAM on interesting cases (high/low confidence, correct/incorrect)

    Args:
        explainer: FraudDetectionExplainer instance
        test_loader: Test data loader
    """
    print("\nFinding interesting cases for detailed Grad-CAM analysis...")
    print("=" * 60)

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    explainer.model.eval()

    # Collect interesting cases
    cases = {
        'high_conf_correct': [],
        'high_conf_incorrect': [],
        'low_conf_correct': [],
        'tampered_detected': [],
        'tampered_missed': []
    }

    with torch.no_grad():
        for batch_images, batch_labels in test_loader:
            batch_images = batch_images.to(device)
            batch_labels = batch_labels.to(device)

            outputs = explainer.model(batch_images)
            probs = F.softmax(outputs, dim=1)
            _, predicted = torch.max(outputs, 1)

            for i in range(len(batch_images)):
                confidence = probs[i].max().item()
                true_label = batch_labels[i].item()
                pred_label = predicted[i].item()
                is_correct = pred_label == true_label

                case_info = {
                    'image': batch_images[i].cpu(),
                    'true_label': true_label,
                    'predicted': pred_label,
                    'confidence': confidence,
                    'is_correct': is_correct
                }

                # Categorize cases
                if confidence > 0.9 and is_correct and len(cases['high_conf_correct']) < 2:
                    cases['high_conf_correct'].append(case_info)
                elif confidence > 0.9 and not is_correct and len(cases['high_conf_incorrect']) < 2:
                    cases['high_conf_incorrect'].append(case_info)
                elif confidence < 0.7 and is_correct and len(cases['low_conf_correct']) < 2:
                    cases['low_conf_correct'].append(case_info)
                elif true_label == 1 and pred_label == 1 and len(cases['tampered_detected']) < 2:
                    cases['tampered_detected'].append(case_info)
                elif true_label == 1 and pred_label == 0 and len(cases['tampered_missed']) < 2:
                    cases['tampered_missed'].append(case_info)

    # Analyze each category
    class_names = ['Authentic', 'Tampered']
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

    for case_type, case_list in cases.items():
        if case_list:
            print(f"\n{case_type.replace('_', ' ').title()} Cases:")
            print("-" * 50)

            for j, case in enumerate(case_list):
                print(f"\nCase {j+1}: True={class_names[case['true_label']]}, "
                      f"Pred={class_names[case['predicted']]} (conf: {case['confidence']:.3f})")

                # Generate explanation
                result = explainer.explain_prediction(
                    case['image'],
                    true_label=case['true_label']
                )

                # Save visualization
                save_path = f'/content/drive/MyDrive/fraud_detection/outputs/gradcam_{case_type}_{j+1}_{timestamp}.png'
                visualize_explanation(result, save_path=save_path)

                # Analyze focus
                analyze_tampering_focus(result)

# Create explainer instance
print("Initializing Grad-CAM explainer for fraud detection model...")
explainer = create_fraud_explainer('20250603_145722')

# Grad-CAM Results

In [ ]:
# Test on random sample images
test_gradcam_samples(explainer, test_loader, num_samples=6)

# Grad-CAM Analysis

In [ ]:
# Test on specific interesting cases
test_gradcam_interesting_cases(explainer, test_loader)

In [ ]:
def compare_class_explanations(explainer, image_tensor, true_label=None):
    """
    Compare Grad-CAM explanations for both classes on the same image

    Args:
        explainer: FraudDetectionExplainer instance
        image_tensor: Input image tensor
        true_label: Ground truth label
    """
    print("\nComparing explanations for both classes on the same image...")

    # Get explanations for both classes
    authentic_result = explainer.explain_prediction(image_tensor, true_label, target_class=0)
    tampered_result = explainer.explain_prediction(image_tensor, true_label, target_class=1)

    # Create comparison visualization
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))

    # Original image (shown twice for reference)
    for row in range(2):
        axes[row, 0].imshow(authentic_result['visualization']['original_image'])
        axes[row, 0].set_title('Original Image')
        axes[row, 0].axis('off')

    # Authentic explanation
    axes[0, 1].imshow(authentic_result['visualization']['heatmap'], cmap='jet')
    axes[0, 1].set_title(f"Authentic Class Explanation\n(Intensity: {authentic_result['explanation']['heatmap_intensity']:.3f})")
    axes[0, 1].axis('off')

    axes[0, 2].imshow(authentic_result['visualization']['cam_overlay'])
    auth_conf = authentic_result['prediction']['probabilities']['authentic']
    axes[0, 2].set_title(f"Authentic Overlay\n(Prob: {auth_conf:.3f})")
    axes[0, 2].axis('off')

    # Tampered explanation
    axes[1, 1].imshow(tampered_result['visualization']['heatmap'], cmap='jet')
    axes[1, 1].set_title(f"Tampered Class Explanation\n(Intensity: {tampered_result['explanation']['heatmap_intensity']:.3f})")
    axes[1, 1].axis('off')

    axes[1, 2].imshow(tampered_result['visualization']['cam_overlay'])
    tamp_conf = tampered_result['prediction']['probabilities']['tampered']
    axes[1, 2].set_title(f"Tampered Overlay\n(Prob: {tamp_conf:.3f})")
    axes[1, 2].axis('off')

    plt.tight_layout()

    # Save comparison
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    save_path = f'/content/drive/MyDrive/fraud_detection/outputs/gradcam_comparison_{timestamp}.png'
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.show()

    print(f"Class comparison saved to: {save_path}")

    # Analyze differences
    auth_intensity = authentic_result['explanation']['heatmap_intensity']
    tamp_intensity = tampered_result['explanation']['heatmap_intensity']

    print(f"\nClass Explanation Comparison:")
    print(f"Authentic explanation intensity: {auth_intensity:.3f}")
    print(f"Tampered explanation intensity: {tamp_intensity:.3f}")
    print(f"Intensity difference: {abs(tamp_intensity - auth_intensity):.3f}")

    if tamp_intensity > auth_intensity:
        print("Model shows stronger activation patterns when explaining tampering")
    else:
        print("Model shows stronger activation patterns when explaining authenticity")

# Test comparison on an interesting image
print("Running comparative class explanation analysis...")
images, labels = next(iter(test_loader))
compare_class_explanations(explainer, images[0], labels[0].item())

print("\nGrad-CAM analysis complete!")
print("All visualizations saved to /content/drive/MyDrive/fraud_detection/outputs/")